This notebook is for the data complexity labelling and preprocessing

In [ ]:
# Install missing dependency
!pip install textstat datasets

import json
import random
import pandas as pd
import textstat  # Import textstat for readability scoring
from datasets import Dataset, load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration


In [ ]:
# Define paths
DATA_DIR = "data"
RAW_DATA_PATH = os.path.join(DATA_DIR, "ai_myths_expanded_1000.json")
PROCESSED_DATA_PATH = os.path.join(DATA_DIR, "ai_myths_1000_complexity.json")

# Load the base dataset
with open(RAW_DATA_PATH, "r", encoding="utf-8") as f:
    dataset = json.load(f)

# Process each example to compute Flesch Reading Ease score and assign Complexity label.
# Higher readability score means easier (Simple); lower means more complex.
for example in dataset:
    readability = textstat.flesch_reading_ease(example["Debunked Fact"])
    example["Readability"] = readability
    example["Complexity"] = "Simple" if readability >= 60 else "Complex"

# Sort the dataset so that the easiest (highest readability) examples come first.
dataset_sorted = sorted(dataset, key=lambda x: x["Readability"], reverse=True)

# Convert to Pandas DataFrame
df = pd.DataFrame(dataset_sorted)

# Save processed dataset
df.to_json(PROCESSED_DATA_PATH, orient="records", indent=4)

print(f"Dataset processed and saved at {PROCESSED_DATA_PATH}")

# Convert to Hugging Face Dataset
dataset_hf = Dataset.from_pandas(df)

In [ ]:
## Load Pre-trained T5 Model & Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base", model_max_length=128)
model = T5ForConditionalGeneration.from_pretrained("t5-base")

## Tokenize Data
def preprocess_function(examples):
    inputs = ["debunk: " + myth for myth in examples["Myth"]]
    targets = [fact for fact in examples["Debunked Fact"]]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids

    model_inputs["labels"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels
    ]

    return model_inputs


# Apply tokenization
tokenized_datasets = dataset_hf.map(preprocess_function, batched=True)

# Split the tokenized dataset into Simple and Complex subsets.
simple_dataset = tokenized_datasets.select(range(500))
complex_dataset = tokenized_datasets.select(range(500, 1000))